In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# 데이터 불러오기
store_data = pd.read_csv("../data/store_data.csv")
review_data = pd.read_csv("../data/맛_서비스_양_긍부정.csv")


In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc

# ---- 한글 폰트 설정 (깨짐 방지) ----
rc("font", family="Malgun Gothic")
plt.rcParams["axes.unicode_minus"] = False

# ---- 데이터 불러오기 ----
store_data = pd.read_csv("../data/store_data.csv")
review_data = pd.read_csv("../data/\ub9db_\uc11c\ube44\uc2a4_\uc591_\uae0d\ubd80\uc815.csv")

# ---- 제외할 store_id 항목 ----
excluded_ids = [10, 15, 19, 43]

# ---- 카테고리 매핑 ----
def map_category(cat):
    category_mapping = {
        '한식': ['한식', '곰탕,설렁탕', '돼지고기구이', '백반,가정식', '육류,고기요리', '순대,순댓국'],
        '중식': ['중식당'],
        '일식': ['일식당', '생선회', '초밥,롤'],
        '양식': ['이탈리아음식', '햄버거'],
    }
    for key, values in category_mapping.items():
        if cat in values:
            return key
    return '기타'

store_data['main_category'] = store_data['category'].apply(map_category)
store_data = store_data[store_data['main_category'].isin(['한식', '중식', '일식', '양식'])]
store_data = store_data[~store_data['store_id'].isin(excluded_ids)]

# ---- 카테고리별 가격 점수 평균 계산 ----
def calculate_category_price_scores(df, avg_prices):
    price_scores = {}
    for category in df['main_category'].unique():
        scores = []
        for _, row in df[df['main_category'] == category].iterrows():
            if avg_prices[category] > 0 and row['price_range'] > 0:
                score = max(0, 1 - abs(row['price_range'] - avg_prices[category]) / avg_prices[category])
                scores.append(score)
        price_scores[category] = np.mean(scores) if scores else 0
    return price_scores

category_price_scores = calculate_category_price_scores(store_data, category_avg_price)

# ---- 방사형 차트 그리기 ----
def plot_radar_chart(store_id, store_name, main_category, store_df, review_df, avg_prices, avg_review, avg_price_scores, save_path):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # ---- 데이터 준비 ----
    store_info = store_df[store_df['store_id'] == store_id].iloc[0]
    review_info = review_df[review_df['store_name'] == store_name].iloc[0]

    # ---- 가격 점수 정규화 ----
    price_score = 0
    if avg_prices[main_category] > 0 and store_info['price_range'] > 0:
        price_score = max(0, 1 - abs(store_info['price_range'] - avg_prices[main_category]) / avg_prices[main_category])

    # ---- 개별 가게 값 ----
    categories = ['맛', '서비스', '양', '별점', '가격']
    values = [
        review_info['final_taste_positive'],
        review_info['final_service_positive'],
        review_info['final_quantity_positive'],
        store_info['weighted_rating'] / 5,
        price_score
    ]
    
    # ---- 값 누락 체크 ----
    if any(v <= 0 for v in values):
        print(f"{store_name} ({store_id}) 데이터에 누락된 값이 있어 건너뛰었습니다.")
        return

    # ---- 카테고리 평균 값 ----
    category_values = [
        avg_review.loc[main_category, 'final_taste_positive'],
        avg_review.loc[main_category, 'final_service_positive'],
        avg_review.loc[main_category, 'final_quantity_positive'],
        store_df[store_df['main_category'] == main_category]['weighted_rating'].mean() / 5,
        avg_price_scores[main_category]
    ]

    # ---- 값 리스트 준비 ----
    values += values[:1]
    category_values += category_values[:1]
    angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
    angles += angles[:1]
    
    # ---- 방사형 차트 그리기 ----
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.plot(angles, values, linewidth=2, linestyle='solid', label=store_name, color='#1f77b4')
    ax.fill(angles, values, alpha=0.25, color='#1f77b4')
    ax.plot(angles, category_values, linewidth=2, linestyle='dashed', label=f"{main_category} 평균", color='red')

    # ---- 수치 표시 ----
    for angle, value in zip(angles, values):
        ax.text(angle, value - 0.05, f"{value:.2f}", ha='center', va='center', fontsize=9)

    # ---- 설정 및 저장 ----
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    ax.set_thetagrids(np.degrees(angles[:-1]), categories)
    ax.set_ylim(0, 1)
    ax.spines['polar'].set_visible(False)
    ax.set_title(f"{store_name} 리뷰 분석", size=15, pad=20)
    ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1), fontsize=10)
    ax.set_yticklabels([])

    file_path = os.path.join(save_path, f"{store_id}_radar_chart.png")
    plt.savefig(file_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"{store_name} ({store_id}) 차트가 저장되었습니다: {file_path}")

# ---- 방사형 차트 생성 ----
valid_stores = store_data[store_data['store_name'].isin(review_data['store_name'])]
for _, row in valid_stores.iterrows():
    plot_radar_chart(
        row['store_id'], row['store_name'], row['main_category'], 
        store_data, review_data, category_avg_price, category_avg_review, category_price_scores,
        save_path="C:/study/fintech/final_project/jupyter/방사형차트"
    )


고가네철판불백 (2) 차트가 저장되었습니다: C:/study/fintech/final_project/jupyter/방사형차트\2_radar_chart.png
곰바로곰탕 (6) 차트가 저장되었습니다: C:/study/fintech/final_project/jupyter/방사형차트\6_radar_chart.png
곰작골나주곰탕_역삼초교점 (7) 차트가 저장되었습니다: C:/study/fintech/final_project/jupyter/방사형차트\7_radar_chart.png
교토일식 (8) 차트가 저장되었습니다: C:/study/fintech/final_project/jupyter/방사형차트\8_radar_chart.png
남강매점 (13) 차트가 저장되었습니다: C:/study/fintech/final_project/jupyter/방사형차트\13_radar_chart.png
노브랜드버거_뱅뱅사거리점 (14) 차트가 저장되었습니다: C:/study/fintech/final_project/jupyter/방사형차트\14_radar_chart.png
두두돼지불백 (20) 차트가 저장되었습니다: C:/study/fintech/final_project/jupyter/방사형차트\20_radar_chart.png
두메_도곡점 (21) 차트가 저장되었습니다: C:/study/fintech/final_project/jupyter/방사형차트\21_radar_chart.png
라망드쉐프 (25) 차트가 저장되었습니다: C:/study/fintech/final_project/jupyter/방사형차트\25_radar_chart.png
리첸 (26) 차트가 저장되었습니다: C:/study/fintech/final_project/jupyter/방사형차트\26_radar_chart.png
만복회해산물_뱅뱅사거리점 (27) 데이터에 누락된 값이 있어 건너뛰었습니다.
맥도날드_서초뱅뱅점 (28) 데이터에 누락된 값이 있어 건너뛰었습니다.
버드나무집_서초본점 (36) 데이터에 누락된 값이 있어